In [1]:
import pandas as pd
import numpy as np 
from scipy import stats
import seaborn as sns
sns.set(color_codes=True)

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import ConfusionMatrixDisplay as CMD
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [2]:
def split_df(df, pheno):
    x_tr, x_te, y_tr, y_te = train_test_split(df,
                                          pheno, 
                                          test_size=0.3, 
                                          random_state=123)
    x_tr = StandardScaler().fit_transform(x_tr)
    x_te = StandardScaler().fit_transform(x_te)
    
    return x_tr, x_te, y_tr, y_te

def fit_model(x_tr, y_tr):
    
    
    param_grid = [
            {'penalty': ['l1','l2',], 
             'C': np.logspace(-3,3,7),
             'max_iter': [1000],
             'solver': ['liblinear']},
            {'penalty': ['l2'], 
             'C': np.logspace(-3,3,7),
             'max_iter': [500],
             'solver': ['newton-cg']},
             ]
    
    
    log_reg = LR(random_state=123)
    log_reg_cv = GridSearchCV(log_reg, 
                              param_grid=param_grid, 
                              scoring='f1_macro',
                              n_jobs=-1,
                              cv=5)
       

    model = log_reg_cv.fit(x_tr, y_tr)
    best_params = model.best_params_
    
    logreg = LR(C=best_params['C'],
                max_iter = best_params['max_iter'],
                penalty = best_params['penalty'],
                solver = best_params['solver'])
    model = logreg.fit(x_tr, y_tr)
    
    return model, best_params
def accuracy_stats(x_te, 
                   y_te, 
                   y_pred,
                   model,
                   multi_class=False):
    
    res = {}
    res['confusion matrix'] =confusion_matrix(y_te, y_pred)
    res['f1'] = f1_score(y_te, y_pred, average='weighted')
    if multi_class == False:
        res['ROC AUC'] = roc_auc_score(y_te, model.predict_proba(x_te)[:,1])  
    
    return res
def print_scores(model, res, multi_class=False):
        print("-------------------------------------")
        print("Model fitting complete")
        print("-------------------------------------")
        
        if multi_class == False:
            TN, FP, FN, TP = res['confusion matrix'].ravel()
            print('True Positive(TP)  = ', TP)
            print('False Positive(FP) = ', FP)
            print('True Negative(TN)  = ', TN)
            print('False Negative(FN) = ', FN)
            print("-------------------------------------")
        
            print("ROC AUC score of fitted model on test data: ", res['ROC AUC'])
            print("-------------------------------------")
        
        print("F1 score of fitted model on test data: ", res['f1'])
        print("-------------------------------------")
        

In [5]:
path = '/dccstor/boseukb/CuNA/data/cuna_input_heart_failure.csv'
df = pd.read_csv(path, sep="\t")
# df.index=df['PATNO']

# y = df['Dx']
# df.drop(['PATNO','Dx', 'DxPro','DYSKPRES'], axis=1, inplace=True)
# assert df.shape[0] == len(y)
# print('X and y dimensions match!\n')
# df = df[df.columns[:5]]
print("Number of individuals: ", df.shape[0])
print("Number of features: ", df.shape[1])

Number of individuals:  58
Number of features:  133


In [20]:
df.head()

,B cells naive,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells regulatory (Tregs),NK cells resting,Monocytes,Macrophages M2,...,ANKRD36B,SMIM24,PLXNB2,METTL21EP,LOC613206,TLR7,ME1,OR10G4,RAPH1,ZP3
0,0.000000,0.000000,0.106755,0.051565,0.078719,0.000000,0.022818,0.096311,0.170335,0.005556,...,7.445499,7.640730,8.712936,4.684011,4.997265,7.507433,5.750543,4.376022,5.632643,5.809710
1,0.058243,0.005087,0.073136,0.151708,0.023674,0.002419,0.000397,0.172571,0.134780,0.005884,...,7.700520,7.738650,9.125528,5.287033,5.162620,7.803266,5.197506,5.011310,6.685016,6.484208
2,0.022525,0.000000,0.121288,0.014656,0.099991,0.000000,0.024720,0.118176,0.144301,0.007580,...,7.628441,6.829061,8.686001,5.207176,5.722934,7.915297,5.973175,5.108977,5.359273,6.369091
3,0.064221,0.000000,0.078102,0.146272,0.002452,0.000000,0.000000,0.204669,0.054148,0.011717,...,7.814651,6.297211,8.703609,5.602468,5.616184,7.777238,5.589794,4.706966,5.886183,5.801833
4,0.040828,0.002891,0.024794,0.068022,0.023230,0.009039,0.013051,0.068168,0.181226,0.000000,...,7.749688,7.349623,8.688090,5.133403,5.156588,8.271997,5.374964,4.549581,5.421573,5.581598


In [26]:
clinical = pd.read_csv("/dccstor/boseukb/CuNA/data/clinical_multiomics_HFhospitalizations_2023-10-23.tsv", sep="\t")
y = clinical.hospitalizations
clinical.drop('hospitalizations', axis=1, inplace=True)
binary_cols = [col for col in clinical if np.isin(clinical[col].unique(), ['Y', 'N']).all()]
y = y.map({'Yes':1, 'No':0})
for col in binary_cols: 
     clinical[col] = clinical[col].map({'Y':1, 'N': 0})
alldf = pd.concat([df,clinical], 
                  axis=1,
                  ignore_index=True)
alldf = pd.concat([alldf,y], axis=1, ignore_index=True)
alldf.columns = list(df.columns) + list(clinical.columns) + ["hospitalization"]


In [27]:
alldf.to_csv("/dccstor/boseukb/CuNA/data/HF4cuna.csv", sep='\t', index=False)

In [28]:
x_tr, x_te, y_tr, y_te = split_df(df,y)

In [29]:
model, best_params = fit_model(x_tr, y_tr)

In [30]:
print("Best performing logistic regression model on training data: ", best_params)
y_pred = model.predict(x_te)  
res = accuracy_stats(x_te, y_te, y_pred, model)  
print_scores(model, res)

Best performing logistic regression model on training data:  {'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
-------------------------------------
Model fitting complete
-------------------------------------
True Positive(TP)  =  128
False Positive(FP) =  17
True Negative(TN)  =  53
False Negative(FN) =  23
-------------------------------------
ROC AUC score of fitted model on test data:  0.9000946073793756
-------------------------------------
F1 score of fitted model on test data:  0.8208891963928227
-------------------------------------
